In [1]:
import numpy as np

In [39]:
A = np.array([
    [1, -1, 2, -1],
    [2, 0, 3, 1],
    [1, 1, 3, -1],
    [2, 1, 5, -2]
])

b = np.array([1, 4, 2, 3])

# 1. Метод Гаусса

In [40]:
def triangle_view(matrix: np.ndarray, eps: float = 1e-7) -> np.ndarray:
    A = matrix.copy()
    N = A.shape[0]
    for i in range(N - 1):
        if abs(A[i, i]) < eps: # если коэффициент - нуль
            for k in range(i + 1, N):
                if abs(A[k, i]) > eps:
                    A[i], A[k] = A[k], A[i]
                    break
        for j in range(i + 1, N):
            d = A[j, i] / A[i, i]
            A[j] -= - d * A[i]
    return A


In [45]:
def get_roots(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    

In [47]:
def gauss_method(A: np.ndarray, b: np.ndarray, eps: float = 1e-7) -> np.ndarray:
    forward_pass = triangle_view(np.c_[A, b], eps)
    return get_roots(forward_pass[:, :-1], forward_pass[:, -1])

gauss_method(A, b)

(array([[ 1, -1,  2],
        [ 2,  0,  3],
        [ 1,  1,  3],
        [ 2,  1,  5]]),
 array([-1,  1, -1, -2]))